# Project 3 Amusement Parks in Europe 

### Goals:

1) Get basic information on top 10 Amusement parks in Europe (python - in jupyter notebook)

2) Create a database (MS SQL Server)

3) Make an visualisation (Power BI)

4) Write a script for "search engine", that would provide information about the amusement parks (python - in pycharm)

In [20]:
#scraping data from wikipedia

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd


themeparks = ['Energylandia', 'Europa-Park', 'Walibi_Belgium', 'Mirabilandia_(Italy)', 'Tivoli_(Copenhagen)', 
              'Disneyland_Paris','Liseberg', 'Efteling', 'Alton_Towers', 'PortAventura_World'
             ]
url_wiki = 'https://en.wikipedia.org/wiki/'

columns = ['Name', 'Location', 'Coordinates', 'Opened',
           'Visitors per annum', 'Area', 'Total', 'Roller coasters', 'Water rides', 'Website',
          ]

def main(list_themeparks, url, list_columns):
    raw_dict = dict()
    for park in list_themeparks:
        full_url = url + park
        rows = get_infobox_from_wiki(full_url)
        keys_list = get_keys(rows)
        vals_list = get_values(rows)
        temp_dict = get_dict_from_lists(keys_list, vals_list)
        raw_dict[park] = temp_dict
    fin_dict = make_fin_dict(list_columns)
    fin_dict = add_name_column(raw_dict, fin_dict)
    fin_dict = add_rest_columns(raw_dict, fin_dict)
    df = make_df_from_dict(fin_dict)
    return df

def get_infobox_from_wiki(url):
    response = rq.get(url)
    soup = bs(response.text, 'html.parser')
    table = soup.find('table', {'class': 'infobox vcard'})
    rows = table.find_all('tr')
    return rows

def get_keys(rows):
    keys_list = []
    for i in rows:
        try:
            a = i.find('th').get_text()
            keys_list.append(a)
        except AttributeError:
            a = ''
            keys_list.append(a)
    return keys_list

def get_values(rows):
    vals_list = []
    for i in rows:
        try:
            a = i.find('td').get_text()
            vals_list.append(a)
        except AttributeError:
            a = ''
            vals_list.append(a)
    return vals_list

def get_dict_from_lists(list1, list2):
    temp_dict = dict()
    for i in range(0, len(list1)):
        temp_dict[list1[i]] = list2[i]
    return temp_dict

def make_fin_dict(list_columns):
    fin_dict = {key: list() for key in list_columns}
    return fin_dict

def add_name_column(raw_dict, fin_dict):
    for key in raw_dict:
        if 'Alton' not in key:
            try:
                name = key.split('_')[0]
            except:
                name = key
        else:
            name = key.replace('_', '')
        fin_dict['Name'].append(name)
    return fin_dict

def add_rest_columns(raw_dict, fin_dict):
    for col in fin_dict:
        for dict in raw_dict.values():
            if col != 'Name':
                if col in dict.keys():
                    fin_dict[col].append(dict[col])
                else:
                    fin_dict[col].append(None)
    return fin_dict

def make_df_from_dict(fin_dict):
    df_full = pd.DataFrame({key:pd.Series(value) for key, value in fin_dict.items()})
    return df_full


df_parks = main(themeparks, url_wiki, columns)
df_parks.head()

,Name,Location,Coordinates,Opened,Visitors per annum,Area,Total,Roller coasters,Water rides,Website
0,Energylandia,"Zator, Lesser Poland, Poland",50°00′00″N 19°24′33″E﻿ / ﻿50.000115°N 19.40907...,14 July 2014 (2014-07-14),"1,600,000 (2019)[1]",35 hectares (86 acres),52,15,None,https://energylandia.pl/en
1,Europa-Park,"Europa-Park-Straße 277977 Rust, Baden-Württemb...",48°16′06″N 7°43′15″E﻿ / ﻿48.26833°N 7.72083°E﻿...,12 July 1975; 45 years ago (12 July 1975),5.75 million (2019),"950,000 m2 (10,225,715 sq ft) (parks, hotels, ...",72,13,11,www.europapark.de
2,Walibi,"Wavre, Walloon Brabant, Belgium",50°41′55″N 4°35′26″E﻿ / ﻿50.69861°N 4.59056°E﻿...,"July 26, 1975",None,None,±50,9,3,http://www.walibi.com/belgium/be-en
3,Mirabilandia,"Savio, Ravenna (RA), Emilia-Romagna, Italy",44°20′14″N 12°15′48″E﻿ / ﻿44.33722°N 12.26333°...,"July 4, 1992",1.4 million (2016),40 ha (99 acres),39,7,7,"mirabilandia.it(English, ItaIian, German)"
4,Tivoli,"Copenhagen, Denmark",55°40′25″N 12°34′06″E﻿ / ﻿55.67361°N 12.56833°...,15 August 1843; 177 years ago (1843-08-15),"4,6 million in 2016[2]",None,25,4,2,Tivoli.dk
5,Disneyland,"Chessy, France",48°52′7.39″N 2°46′54.48″E﻿ / ﻿48.8687194°N 2.7...,12 April 1992; 28 years ago (1992-04-12),14.8 million (2017),"19.425 km2 (4,800 acres)",None,None,None,disneylandparis.com
6,Liseberg,"Gothenburg, Sweden",57°41′44″N 11°59′24″E﻿ / ﻿57.69556°N 11.99000°...,8 May 1923,"3,055,000(2018)[1] Halloween: 185,000 (2016)[...",17 hectares (42 acres),37,6,2,Liseberg.com
7,Efteling,"Kaatsheuvel, North Brabant, Netherlands",51°39′01″N 5°02′53″E﻿ / ﻿51.65028°N 5.04806°E﻿...,"May 31, 1952; 68 years ago (1952-05-31)",5.26 million in 2019[1],"72 ha (720,000 m2) : the park ; 276 ha (2,760,...",36,6,4,www.efteling.com
8,AltonTowers,"Alton, Staffordshire, England",52°59′15″N 1°53′27″W﻿ / ﻿52.98750°N 1.89083°W﻿...,13 April 1860 (1860-04-13) (as country estate)...,2.13 million (2019),910 acres (370 ha) (total combined resort area...,37,10,3,www.AltonTowers.com
9,PortAventura,"Salou and Vila-seca, Catalonia, Spain",41°05′13″N 1°09′26″E﻿ / ﻿41.086839°N 1.157272°...,13 June 2002; 18 years ago (2002-06-13) (resor...,5.2 million (2019)[2],119 ha [3]294 acres,53,9,5,Official website


In [142]:
#first cleaning of the data

import re

def col_country(string):
    country = string.split(',')[-1]
    return country

def col_latitude(string):
    coord_decimal = string.split('/')[-1]
    latitude = float(coord_decimal.split(';')[0])
    return latitude

def col_longitude(string):
    coord_decimal = string.split('/')[-1]
    longitude = float(coord_decimal.split(';')[-1])
    return longitude

def col_opened(string):
    day = re.findall('\d{1,2}', string)[0]
    month = re.findall('\w+[a-zA-Z]\s', string)[0]
    year = re.findall('\d{4}', string)[0]
    date = pd.to_datetime('{}-{}-{}'.format(day, month, year))
    return date

def col_visitors(string):
    visit = 0
    try:
        visit = re.findall('\d{1}[.]\d{1,2}', string)[0]
    except:
        try:
            visit = re.findall('\d{1}[,]\d{1,2}', string)[0].replace(',', '.')
        except:
            pass
    return float(visit)
                 
def col_area_acres(string):
    area = None
    try:
        area_str = re.findall('\d{1,3}\sacres', string)[0]
        area = int(area_str.split(' ')[0])
    except:
        try:
            area_m2_str = re.findall('\d{3}[,]\d{3}\sm2', string)[0].replace(',', '')
            area_m2 = int(area_m2_str.split()[0])
            area = round(area_m2*0.000247)
        except:
            pass                   
    return area

def col_attractions(string):
    attractions = None
    try:
        attractions = int(re.findall('\d{1,2}', string)[0])
    except:
        pass
    return attractions

def col_rollers(string):
    rollers = None
    try:
        rollers = int(string)
    except:
        pass
    return rollers

def col_waterrides(string):
    waterrides = None
    try:
        waterrides = int(string)
    except:
        pass
    return waterrides

def col_website(string):
    web = None
    try:
        web = re.findall('www[.]\w+[.]\w{2,3}', string)[0]
    except:
        try:
            web_name = re.findall('\w+[.]\w{2,3}', string)[0]
            web = 'www.' + web_name
        except:
            pass
    return web

df_parks_fin = pd.DataFrame()
df_parks_fin['name'] = df_parks['Name']
df_parks_fin['country'] = df_parks['Location'].apply(col_country)
df_parks_fin['latitude'] = df_parks['Coordinates'].apply(col_latitude)
df_parks_fin['longitude'] = df_parks['Coordinates'].apply(col_longitude)
df_parks_fin['opening_date'] = df_parks['Opened'].apply(col_opened)
df_parks_fin['visitors_per_annum'] = df_parks['Visitors per annum'].apply(col_visitors)
df_parks_fin['area_in_acres'] = df_parks['Area'].apply(col_area_acres)
df_parks_fin['num_attractions'] = df_parks['Total'].apply(col_attractions)
df_parks_fin['num_rollers_coasters'] = df_parks['Roller coasters'].apply(col_rollers)
df_parks_fin['num_water_rides'] = df_parks['Water rides'].apply(col_waterrides)
df_parks_fin['website'] = df_parks['Website'].apply(col_website)
df_parks_fin.set_index('name', inplace=True)
df_parks_fin

,name,country,latitude,longitude,opening_date,visitors_per_annum,area_in_acres,num_attractions,num_rollers_coasters,num_water_rides,website
0,Energylandia,Poland,50.000115,19.409078,2014-07-14,1.60,86.0,52.0,15.0,NaN,www.energylandia.pl
1,Europa-Park,Germany,48.268330,7.720830,1975-07-12,5.75,235.0,72.0,13.0,11.0,www.europapark.de
2,Walibi,Belgium,50.698610,4.590560,1975-07-26,0.00,NaN,50.0,9.0,3.0,www.walibi.com
3,Mirabilandia,Italy,44.337220,12.263330,1992-07-04,1.40,99.0,39.0,7.0,7.0,www.mirabilandia.it
4,Tivoli,Denmark,55.673610,12.568330,1843-08-15,4.60,NaN,25.0,4.0,2.0,www.Tivoli.dk
5,Disneyland,France,48.868719,2.781800,1992-04-12,4.80,800.0,NaN,NaN,NaN,www.disneylandparis.com
6,Liseberg,Sweden,57.695560,11.990000,1923-05-08,3.05,42.0,37.0,6.0,2.0,www.Liseberg.com
7,Efteling,Netherlands,51.650280,5.048060,1952-05-31,5.26,178.0,36.0,6.0,4.0,www.efteling.com
8,AltonTowers,England,52.987500,-1.890830,1860-04-13,2.13,910.0,37.0,10.0,3.0,www.AltonTowers.com
9,PortAventura,Spain,41.086839,1.157272,2002-06-13,5.20,294.0,53.0,9.0,5.0,None


In [1]:
#scraping data from themeparks-eu.com and klook.com

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd

parks = ['tivoli-gardens', 'europa-park', 'disneyland-park', 'efteling', 'alton-towers', 'portaventura-park', 'mirabilandia', 'energylandia', 'walibi-belgium', 'liseberg']


url = 'https://www.themeparks-eu.com/parks/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}


df_highs = pd.DataFrame()
df_phone = pd.DataFrame()
df_parks2 = pd.DataFrame()
for park in parks:
    full_url = url + park
    response1 = rq.get(full_url, headers=headers)
    soup = bs(response1.text, 'html.parser')
    try:
        h3_part = soup.find('h3', string='Highlights').find_next_siblings()[0]
        highlights = [h.get_text() for h in (h3_part.find_all('li'))]
        new_df_highs = pd.DataFrame(highlights)
        new_df_highs['name'] = park
        df_highs = pd.concat([df_highs, new_df_highs])
        phonepart = soup.find('div', {'class': 'pos-contact'}).find('li').get_text()
        phone = phonepart.split(':')[-1]
        new_row_ph = {'name': park, 'phone': phone}
        df_phone = df_phone.append(new_row_ph, ignore_index=True)
        try:
            link = soup.find('div', {'class': 'element element-link'}).find('a')['href']
            response2 = rq.get(link, headers=headers)
            soup = bs(response2.text, 'html.parser')
            review = soup.find('span', {'class': 'activity-core'}).get_text()
            price = soup.find('div', {'class': 'price-package'}).find('b').get_text()
        except:
            review = None
            price = None
        finally:
            new_row = {'name': park, 'review': review, 'price_from': price}
            df_parks2 = df_parks2.append(new_row, ignore_index=True)
    except:
        print('chyba scrapingu')
        pass

df_phone
df_highs
df_parks2

,name,price_from,review
0,tivoli-gardens,€ 18.15,4.7
1,europa-park,€ 50,4.8
2,disneyland-park,€ 84.9,4.7
3,efteling,€ 45,4.5
4,alton-towers,None,None
5,portaventura-park,None,None
6,mirabilandia,None,None
7,energylandia,None,None
8,walibi-belgium,None,None
9,liseberg,None,None


In [196]:
df_highs.sample(10)

,0,name
5,Dragon - suspended coaster,energylandia
0,Sprookjesbos (Fairytale Forest),efteling
2,Werewolf - wooden coaster,walibi-belgium
2,The Flying Trunk - dark ride,tivoli-gardens
9,Peter Pan's Flight,disneyland-park
9,Carnaval Festival - indoor family ride,efteling
7,Spinball Whizzer - spinning coaster,alton-towers
3,De Vliegende Hollander - combination of a wate...,efteling
5,Himmelskibet - 'star flyer' ride,tivoli-gardens
2,Furius Baco - launched coaster,portaventura-park


In [3]:
df_phone

,name,phone
0,tivoli-gardens,+45 33151001
1,europa-park,+49 (0) 7822 776688
2,disneyland-park,+33 (0) 1 60 30 60 30
3,efteling,+31 (0) 416 537 777
4,alton-towers,+44 (0) 871 222 3330
5,portaventura-park,+34 977 779 090
6,mirabilandia,+39 (0) 544 561156
7,energylandia,+48 33 486 15 00
8,walibi-belgium,+32 (0) 10 42 15 00
9,liseberg,+46 31 400 100


In [5]:
#making a new dataframe with the distances from some czech cities to the amusement parks 
#csv source: https://github.com/33bcdd/souradnice-mest

import geopy.distance

df_souradnice = pd.read_csv('souradnice.csv')
kraj_mesta = ['Brno', 'České Budějovice', 'Hradec Králové', 'Jihlava', 'Karlovy Vary', 'Liberec',
             'Olomouc', 'Ostrava', 'Pardubice', 'Plzeň', 'Praha', 'Ústí nad Labem', 'Zlín'             ]
df_mesta = df_souradnice[df_souradnice['Obec'].isin(kraj_mesta)].iloc[:, [0,7,8]]
df_mesta.set_index('Obec', inplace=True)
col_names = ['parkname', 'cz_city', 'distance_km']

def mk_distance_df(df1, df2, col_names):
    df = pd.DataFrame(columns = col_names)
    for i in df1.index:
        for j in df2.index:  
            distance = get_distance(df1.loc[i,'Latitude'], df1.loc[i,'Longitude'],df2.loc[j,'latitude'], df2.loc[j,'longitude'])
            row = mk_distance_series(j, i, distance, col_names)
            df = df.append(row, ignore_index=True)
    return df
        
def mk_distance_series(col1, col2, col3, index_list):
    row_list = [col1, col2, col3]
    row_series = pd.Series(row_list, index = index_list)
    return row_series
        
def get_distance(lat1,lon1,lat2,lon2):
    coords_1 = (lat1,lon1)
    coords_2 = (lat2,lon2)
    distance = geopy.distance.geodesic(coords_1, coords_2).km
    return distance

df_distance = mk_distance_df(df_mesta, df_parks_fin, col_names)
df_distance.sample(10)
df_mesta.reset_index()

NameError: name 'df_parks_fin' is not defined

In [3]:
# uploading dataframes to the database

import pyodbc
from sqlalchemy import create_engine

server = 'LAPTOP-Q5O9P275\SQLEXPRESS'
database = 'themeparks_work'
engine = create_engine('mssql+pyodbc://@' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')


df_highs.to_sql('parks_highlights_v1', engine, if_exists='replace')
df_parks2.to_sql('parks_info2_v1', engine, if_exists='replace')
df_distance.to_sql('parks_distance_v1', engine, if_exists='replace')
df_phone.to_sql('parks_phones', engine, if_exists='replace')
df_mesta.to_sql('parks_cities', engine, if_exists='replace')